In [3]:
# Let's read the content of the uploaded file to understand the key frame information
file_path = "raw_data/Info_Video_0194.txt"

with open(file_path, 'r') as file:
    key_frame_data = file.read()

# Displaying the first 1000 characters to get an overview of the content
key_frame_data[:1000]


'INFO of 0001:\nroad 0 is at middle_down\nsidewalk 1 is at right_down\nperson 0 is at middle_down\nThe [distance,angle] from person 0 to our dashcam is: [very close,89.97567850454345]\nperson 1 is at middle_down\nThe [distance,angle] from person 1 to our dashcam is: [very close,89.97128033898481]\nperson 2 is at middle_down\nThe [distance,angle] from person 2 to our dashcam is: [very close,-89.98346900030981]\nperson 3 is at left_down\nThe [distance,angle] from person 3 to our dashcam is: [very close,-89.98502153235941]\nperson 4 is at middle_down\nThe [distance,angle] from person 4 to our dashcam is: [very close,-89.98260645390403]\nperson 5 is at left_down\nThe [distance,angle] from person 5 to our dashcam is: [very close,-89.99403606265135]\nPerson 0 is on the road 0, sidewalk 1,his/her bbox is [1148.4463   625.3968  1183.7595   663.28143]\nPerson 1 is on the road 0, sidewalk 1,his/her bbox is [1120.0143  636.9002 1149.0658  672.4008]\nPerson 2 is on the road 0, sidewalk 1,his/her b

In [4]:
# Parsing the key frame data to extract relevant information for risk analysis
from collections import defaultdict

def parse_key_frame_data(data):
    frames = defaultdict(lambda: defaultdict(list))
    current_frame = None
    
    for line in data.split("\n"):
        if line.startswith("INFO of"):
            current_frame = line.split(":")[0].split()[-1]
        elif "person" in line and "is at" in line:
            person_id = int(line.split()[1])
            position = line.split("is at")[-1]
            frames[current_frame]['persons'].append((person_id, position))
        elif "road" in line and "is at" in line:
            road_id = int(line.split()[1])
            position = line.split("is at")[-1]
            frames[current_frame]['roads'].append((road_id, position))
        elif "The [distance,angle]" in line:
            distance, angle = line.split("is: [")[-1].strip("]").split(",")
            frames[current_frame]['persons'][-1] += (distance, float(angle))
    
    return frames

key_frame_info = parse_key_frame_data(key_frame_data)
key_frame_info # Displaying parsed information for analysis


defaultdict(<function __main__.parse_key_frame_data.<locals>.<lambda>()>,
            {'0001': defaultdict(list,
                         {'roads': [(0, ' middle_down')],
                          'persons': [(0,
                            ' middle_down',
                            'very close',
                            89.97567850454345),
                           (1,
                            ' middle_down',
                            'very close',
                            89.97128033898481),
                           (2,
                            ' middle_down',
                            'very close',
                            -89.98346900030981),
                           (3, ' left_down', 'very close', -89.98502153235941),
                           (4,
                            ' middle_down',
                            'very close',
                            -89.98260645390403),
                           (5,
                            ' left_down',
   

In [6]:
# Defining a function to evaluate the risk for each person based on the criteria mentioned above
def evaluate_risk(person_info):
    position = person_info[1]
    distance = person_info[2]
    angle = person_info[3]

    if distance == "very close" and position == "middle_down" and -10 <= angle <= 10:
        return "high"
    elif distance == "very close" and position != "middle_down" and -30 <= angle <= 30:
        return "medium"
    else:
        return "low"

# Applying the risk evaluation function to each person in the key frames
risk_evaluation = defaultdict(dict)
for frame, info in key_frame_info.items():
    for person_info in info['persons']:
        person_id = person_info[0]
        risk_evaluation[frame][f'person {person_id}'] = evaluate_risk(person_info)

# Displaying a subset of the risk evaluation for each person in the key frames
subset_risk_evaluation = {frame: risk for frame, risk in list(risk_evaluation.items())[:5]}
subset_risk_evaluation



{'0001': {'person 0': 'low',
  'person 1': 'low',
  'person 2': 'low',
  'person 3': 'low',
  'person 4': 'low',
  'person 5': 'low'},
 '0002': {'person 0': 'low',
  'person 1': 'low',
  'person 2': 'low',
  'person 3': 'low',
  'person 4': 'low',
  'person 5': 'low'},
 '0003': {'person 0': 'low', 'person 1': 'low', 'person 2': 'low'},
 '0004': {'person 0': 'low',
  'person 1': 'low',
  'person 2': 'low',
  'person 3': 'low',
  'person 4': 'low',
  'person 5': 'low'},
 '0005': {'person 0': 'low', 'person 1': 'low', 'person 2': 'low'}}

In [7]:
# Function to transform the key frame data into natural language sentences
def transform_to_natural_language(data):
    natural_language_data = defaultdict(list)

    for frame, info in data.items():
        # Adding information about roads
        for road_id, position in info.get('roads', []):
            sentence = f"Road {road_id} is at {position.replace('_', ' ')}."
            natural_language_data[frame].append(sentence)

        # Adding information about persons
        for person_info in info.get('persons', []):
            person_id, position, distance, angle = person_info
            sentence = (f"Person {person_id} is at {position.replace('_', ' ')}, "
                        f"{distance}, with an angle of {angle:.2f} degrees to the dashcam.")
            natural_language_data[frame].append(sentence)

    return natural_language_data

# Transforming the key frame info into natural language sentences
natural_language_info = transform_to_natural_language(key_frame_info)

# Displaying a subset of the natural language data for the key frames
subset_natural_language_info = {frame: sentences for frame, sentences in list(natural_language_info.items())[:3]}
subset_natural_language_info


{'0001': ['Road 0 is at  middle down.',
  'Person 0 is at  middle down, very close, with an angle of 89.98 degrees to the dashcam.',
  'Person 1 is at  middle down, very close, with an angle of 89.97 degrees to the dashcam.',
  'Person 2 is at  middle down, very close, with an angle of -89.98 degrees to the dashcam.',
  'Person 3 is at  left down, very close, with an angle of -89.99 degrees to the dashcam.',
  'Person 4 is at  middle down, very close, with an angle of -89.98 degrees to the dashcam.',
  'Person 5 is at  left down, very close, with an angle of -89.99 degrees to the dashcam.'],
 '0002': ['Road 0 is at  middle down.',
  'Person 0 is at  middle down, very close, with an angle of 89.98 degrees to the dashcam.',
  'Person 1 is at  left down, very close, with an angle of -89.98 degrees to the dashcam.',
  'Person 2 is at  middle down, very close, with an angle of 89.98 degrees to the dashcam.',
  'Person 3 is at  left down, very close, with an angle of -89.99 degrees to the da

# Let's read the content of the uploaded file to understand the key frame information
file_path = "raw_data/Info_Video_0194.txt"

with open(file_path, 'r') as file:
    key_frame_data = file.read()

# Displaying the first 1000 characters to get an overview of the content
key_frame_data[:1000]


In [10]:
# Let's read the content of the LLM generated file and load into JSON
file_path = "LLM_data/Result_Video_0194.md"

with open(file_path, 'r') as file:
    result_data = file.read()

# Displaying the first 1000 characters to get an overview of the content
result_data[:1000]


'### Frame 0001 Analysis\n\n**Summary of Key Frame Information:**\n- **Road Position:** The road (road 0) is located in the middle-down region of the frame.\n- **Sidewalk Position:** The sidewalk (sidewalk 1) is located in the right-down region of the frame.\n- **People Positions and Distances:**\n  - **Person 0:** Located at middle-down, very close to the dashcam, with an angle of 89.98 degrees. On road 0, sidewalk 1.\n  - **Person 1:** Located at middle-down, very close to the dashcam, with an angle of 89.97 degrees. On road 0, sidewalk 1.\n  - **Person 2:** Located at middle-down, very close to the dashcam, with an angle of -89.98 degrees. On road 0, sidewalk 1.\n  - **Person 3:** Located at left-down, very close to the dashcam, with an angle of -89.98 degrees. Not on any detected surface.\n  - **Person 4:** Located at middle-down, very close to the dashcam, with an angle of -89.98 degrees. Not on any detected surface.\n  - **Person 5:** Located at left-down, very close to the dashc

In [12]:
import re

def extract_risk_evaluation(text):
    # Splitting the text into sections based on headers
    sections = text.split('###')
    
    # Pattern to match the "Risk Evaluation" section
    risk_pattern = re.compile(r"Risk Evaluation:")
    
    # Dictionary to store the extracted information
    extracted_data = {}
    
    for section in sections:
        # If the section contains "Risk Evaluation", parse it
        if risk_pattern.search(section):
            # Extracting frame number
            frame_number = re.search(r"Frame (\d+)", section).group(1)
            extracted_data[f"Frame{frame_number}"] = {
                "Time": f"{float(frame_number) * 0.5}s", # 2Hz sampling rate
                "Text": "Text summary goes here...", # Example placeholder for text summary
                "Pedestrian": {}
            }
            
            # Extracting risk evaluations for persons
            for match in re.finditer(r"Persons ([\d,-]+): ([A-Za-z]+)", section):
                persons, risk_level = match.groups()
                for person in persons.split(","):
                    person = person.strip()
                    extracted_data[f"Frame{frame_number}"]["Pedestrian"][f"Person {person}"] = {
                        "Risk": risk_level, # Example of storing risk level
                        # Additional information like coordinates can be added here
                    }

    return extracted_data

# Example analysis text
analysis_text = str(result_data)


# Extracting the risk evaluation part and transforming it into the desired JSON format
extracted_data = extract_risk_evaluation(analysis_text)
extracted_data


{'Frame0001': {'Time': '0.5s',
  'Text': 'Text summary goes here...',
  'Pedestrian': {}},
 'Frame0002': {'Time': '1.0s',
  'Text': 'Text summary goes here...',
  'Pedestrian': {}},
 'Frame0003': {'Time': '1.5s',
  'Text': 'Text summary goes here...',
  'Pedestrian': {}},
 'Frame0004': {'Time': '2.0s',
  'Text': 'Text summary goes here...',
  'Pedestrian': {}},
 'Frame0005': {'Time': '2.5s',
  'Text': 'Text summary goes here...',
  'Pedestrian': {}},
 'Frame0006': {'Time': '3.0s',
  'Text': 'Text summary goes here...',
  'Pedestrian': {}},
 'Frame0008': {'Time': '4.0s',
  'Text': 'Text summary goes here...',
  'Pedestrian': {}},
 'Frame0009': {'Time': '4.5s',
  'Text': 'Text summary goes here...',
  'Pedestrian': {}}}

In [13]:
import json
import os

# Create a directory named "JSON_data" if it doesn't exist
os.makedirs('JSON_data', exist_ok=True)

# Define the filename and path
filename = 'risk_evaluation.json'
path = os.path.join('JSON_data', filename)

# Convert the dictionary to a JSON string and write it to the file
with open(path, 'w') as file:
    json.dump(extracted_data, file, indent=4)

print(f"Data has been saved to {path}")


Data has been saved to JSON_data\risk_evaluation.json
